## Workers Answers Data Integration
---

In [1]:
import pandas as pd

In [3]:
df_fig8 =  pd.read_excel('../data/crowd/OptimAL Crowdsourcing Microtasks - Drug Relations (May29).xlsx',sheet_name='Results_Full')
df_amz_1 =  pd.read_excel('../data/crowd/OptimAL Crowdsourcing - Contraindications.xlsx',sheet_name='Results_500')
df_amz_2 =  pd.read_excel('../data/crowd/OptimAL Crowdsourcing - Contraindications.xlsx',sheet_name='Results_500_1')
df_fig8_2 =  pd.read_excel('../data/crowd/OptimALcrowdsourcing_relations_aggregated.xlsx',sheet_name='eachworker')
#df_fig8_2[['disease_name','drug_name','context','Worker Answer','_started_at']] #more than 3 workers assessing

In [4]:
def convert_crowd_df(raw):
    raw.columns = ['disease_name','do_id','drug_name','drug_id','context','worker_label','worker_id','label_id']
    return raw


In [5]:
crowd1 = convert_crowd_df(df_fig8[['disease_name','do_id','drug_name','drug','context','Worker Answer','_worker_id','label_id']])
crowd2 = convert_crowd_df(df_fig8_2[['disease_name','doid','drug_name','drug','context','Worker Answer','_worker_id','label']])
crowd3 = convert_crowd_df(df_amz_1[['Input.annotation','Input.DO_ID','Input.drugname','Input.Drug','Input.context','Answer.category.label','WorkerId','Input.Label_ID']])
crowd4 = convert_crowd_df(df_amz_2[['Input.annotation','Input.DO_ID','Input.drugname','Input.Drug','Input.context','Answer.category.label','WorkerId','Input.Label_ID']])

#### Integrate

In [6]:
crowd_df = pd.DataFrame()
crowd_df = crowd_df.append(crowd1)
crowd_df = crowd_df.append(crowd2)
crowd_df = crowd_df.append(crowd3)
crowd_df = crowd_df.append(crowd4)
#crowd_df = crowd_df.groupby(['disease_name','drug_name','context']).head(3)

In [7]:
crowd_df['worker_label'].unique()

array(['effect', 'indication_treatment', 'contraindication',
       'i_dont_know', 'indication_symptomatic_relief', 'unrelated',
       'unsure', 'Effect', 'Indication: Symptomatic relief',
       'Contraindication', 'Indication: treatment', 'I dont know', nan],
      dtype=object)

---
## Gathering workers tabisna

In [13]:
label_dict={'effect':'Effect',
            'indication_treatment':'Indication: Treatment',
            'Indication: treatment':'Indication: Treatment',
           'contraindication':'Contraindication',
            'indication_symptomatic_relief':'Indication: Symptomatic Relief', 
           'Indication: Symptomatic relief':'Indication: Symptomatic Relief',
           'i_dont_know':'IDK', 'I dont know':'IDK','unrelated':'IDK','unsure':'IDK'}

In [14]:
crowd_df['worker_label'].replace(label_dict, inplace=True)

In [15]:
#crowd_df = pd.merge(crowd_df, labels, how='left', left_on ='worker_label', right_on ='text')
#crowd_df = crowd_df.drop(columns=['text','worker_label']).rename(columns={'relation':'worker_label'})
crowd_df['value'] = 1

In [16]:
crowd_df['worker_label'].unique()

array(['Effect', 'Indication: Treatment', 'Contraindication', 'IDK',
       'Indication: Symptomatic Relief', nan], dtype=object)

In [17]:
len(crowd_df)

8049

## Create a workers consensus table

In [36]:
worker_table = pd.pivot_table(
    crowd_df,
    values='value',
    index=['do_id','drug_id','label_id','context'],
    columns=['worker_label'],
    aggfunc='sum',
    fill_value=0
).reset_index()

In [37]:
worker_table['worker_answers'] = worker_table.sum(axis=1)
worker_table['Contraindication'] = worker_table['Contraindication']/worker_table['worker_answers']
worker_table['Effect'] = worker_table['Effect']/worker_table['worker_answers']
worker_table['IDK'] = worker_table['IDK']/worker_table['worker_answers']
worker_table['Indication: Symptomatic Relief'] = worker_table['Indication: Symptomatic Relief']/worker_table['worker_answers']
worker_table['Indication: Treatment'] = worker_table['Indication: Treatment']/worker_table['worker_answers']
worker_table['inter_agree_workers'] = worker_table.drop(columns='worker_answers').max(axis=1)

In [42]:
worker_table.head()

worker_label,do_id,drug_id,label_id,context,Contraindication,Effect,IDK,Indication: Symptomatic Relief,Indication: Treatment,worker_answers,inter_agree_workers
0,DOID_0050214,DB00536,dfbcb78d-0758-4f15-9d9c-f4048b691d9d.xml,Guanidine is indicated for the reduction of th...,0.071429,0.0,0.000000,0.892857,0.035714,28,0.892857
1,DOID_0050425,DB00268,88db5bac-73c7-4b4f-a207-8f895223d778.xml,"['\n ', 'Parkinson‰Ûªs Dis...",0.000000,0.0,0.000000,0.333333,0.666667,3,0.666667
2,DOID_0050425,DB00413,b5504316-3907-444c-b604-ec9a18cdcd6d.xml,Mirapex (pramipexole dihydrochloride) tablets ...,0.000000,0.0,0.000000,0.000000,1.000000,3,1.000000
3,DOID_0050425,DB05271,a6964835-dc96-4b22-8b64-464a42f6cdf2.xml,NEUPRO is a dopamine agonist indicated for the...,0.000000,0.0,0.000000,0.000000,1.000000,3,1.000000
4,DOID_0050426,DB00495,../DailyMedExtracter/prescription/temp_xml/ba6...,"[None, 'Zidovudine\xa0oral solution\xa0is cont...",0.333333,0.0,0.066667,0.400000,0.200000,15,0.400000


In [52]:
label_list = ['Effect', 'Indication: Treatment', 'Contraindication',
       'Indication: Symptomatic Relief']
print (label_list)
consensus = []
for i in range(len(worker_table)):
    if worker_table.loc[i].inter_agree_workers > .6:
        consensus_label = worker_table.loc[i][label_list].astype(float).idxmax()
        consensus.append(consensus_label)
        continue
    consensus.append('No consensus')

worker_table['worker_consensus'] = consensus

['Effect', 'Indication: Treatment', 'Contraindication', 'Indication: Symptomatic Relief']


In [25]:
#len(worker_table.drop_duplicates(subset=['disease_name',	'drug_name',	'context']))

In [53]:
worker_table.shape

(1791, 12)

In [57]:
worker_table.to_csv('../data/crowd/worker_table.csv', index=False)

In [58]:
worker_table.head(5)

worker_label,do_id,drug_id,label_id,context,Contraindication,Effect,IDK,Indication: Symptomatic Relief,Indication: Treatment,worker_answers,inter_agree_workers,worker_consensus
0,DOID_0050214,DB00536,dfbcb78d-0758-4f15-9d9c-f4048b691d9d.xml,Guanidine is indicated for the reduction of th...,0.071429,0.0,0.000000,0.892857,0.035714,28,0.892857,Indication: Symptomatic Relief
1,DOID_0050425,DB00268,88db5bac-73c7-4b4f-a207-8f895223d778.xml,"['\n ', 'Parkinson‰Ûªs Dis...",0.000000,0.0,0.000000,0.333333,0.666667,3,0.666667,Indication: Treatment
2,DOID_0050425,DB00413,b5504316-3907-444c-b604-ec9a18cdcd6d.xml,Mirapex (pramipexole dihydrochloride) tablets ...,0.000000,0.0,0.000000,0.000000,1.000000,3,1.000000,Indication: Treatment
3,DOID_0050425,DB05271,a6964835-dc96-4b22-8b64-464a42f6cdf2.xml,NEUPRO is a dopamine agonist indicated for the...,0.000000,0.0,0.000000,0.000000,1.000000,3,1.000000,Indication: Treatment
4,DOID_0050426,DB00495,../DailyMedExtracter/prescription/temp_xml/ba6...,"[None, 'Zidovudine\xa0oral solution\xa0is cont...",0.333333,0.0,0.066667,0.400000,0.200000,15,0.400000,No consensus
